In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
import os.path as op
import os

import mne
import hcp
import h5io
import matplotlib.pyplot as plt
import numpy as np
from hcp import io
from hcp import preprocessing as preproc

In [19]:
storage_dir = op.join(op.expanduser('~'),
                      'mne-hcp-data')

hcp_path = op.join(storage_dir, 'HCP')
recordings_path = op.join(storage_dir, 'hcp-meg')
project_path = op.join(storage_dir, 'dynamic-scales')
subjects_dir = op.join(storage_dir, 'hcp-subjects')

subject = '105923'
data_type = 'rest'

In [20]:
def hcp_band_pass(
        raw, fmin, fmax, order=4, notch=True, ftype='butter', n_jobs=1):
    if notch is True:
        raw.notch_filter(
            freqs=np.arange(60, 241, 60), method='iir',
            iir_params=dict(order=order, ftype=ftype))
    raw.filter(fmin, None, n_jobs=n_jobs, method='iir',
               iir_params=dict(order=order, ftype=ftype))
    raw.filter(None, fmin, n_jobs=n_jobs, method='iir',
               iir_params=dict(order=order, ftype=ftype))


def compute_noise_covs(subject, recordings_path, data_type,
                       methods=('empirical',),
                       filter_ranges=((None, None),)):

    data_type = 'noise_empty_room'
 
    raw_noise = io.read_raw_hcp(
        run_index=0, subject=subject, hcp_path=hcp_path, data_type=data_type)
    raw_noise.load_data()
    preproc.apply_ref_correction(raw_noise)
    raw_noise.pick_types(meg=True, ref_meg=False)
    
    for fmin, fmax in filter_ranges:
        this_raw = raw_noise.copy()
        if any([fmin, fmax]):
            hcp_band_pass(this_raw, fmin=fmin, fmax=fmax, notch=False)
            fcomment = 'lp{}-hp-{}'.format(
                fmin if fmin is None else str(fmin).replace('.', 'p'),
                fmax if fmax is None else str(fmax).replace('.', 'p'))
        else:
            fcomment = 'broad'

        noise_covs = mne.compute_raw_covariance(
            this_raw, method=list(methods), return_estimators=True)

        for cov, method in zip(noise_covs, methods):
            cov.save(
                op.join(recordings_path, subject, '%s-%s-%s-cov.fif' % (
                        data_type, method, fcomment)))

compute_noise_covs(subject=subject,
                   recordings_path=recordings_path, data_type=data_type,
                   filter_ranges=[(None, None), (None, 1.5), (8, 12)])

Reading 4D PDF file /home/ubuntu/mne-hcp-data/HCP/105923/unprocessed/MEG/1-Rnoise/4D/c,rfDC...
Creating Neuromag info structure ...
... Setting channel info structure.
... no headshape file supplied, doing nothing.
Currently direct inclusion of 4D weight tables is not supported. For critical use cases please take into account the MNE command "mne_create_comp_data" to include weights as printed out by the 4D "print_table" routine.
Current compensation grade : 0
Reading 0 ... 610353  =      0.000 ...   300.000 secs...
Using up to 1500 segments
Estimating covariance using EMPIRICAL
Done.
Using cross-validation to select the best estimator.
Number of samples used : 610500
[done]
log-likelihood on unseen data (descending order):
   empirical: -1513.996
Using up to 1500 segments
Estimating covariance using EMPIRICAL
Done.
Using cross-validation to select the best estimator.
Number of samples used : 610500
[done]
log-likelihood on unseen data (descending order):
   empirical: -1513.996
High-p

<ipython-input-20-690ac843b615>:8: RuntimeWarning: The default output type is "ba" in 0.13 but will change to "sos" in 0.14
  iir_params=dict(order=order, ftype=ftype))


Low-pass filtering at 8 Hz
The default output type is "ba" in 0.13 but will change to "sos" in 0.14


<ipython-input-20-690ac843b615>:10: RuntimeWarning: The default output type is "ba" in 0.13 but will change to "sos" in 0.14
  iir_params=dict(order=order, ftype=ftype))


Using up to 1500 segments
Estimating covariance using EMPIRICAL
Done.
Using cross-validation to select the best estimator.
Number of samples used : 610500
[done]
log-likelihood on unseen data (descending order):
   empirical: -523.438


In [40]:
out['fwd']['src'][0]

surface = 'white'
add_dist = True
src_type = 'subject_on_fsaverage'
mne.write_forward_solution(
    op.join(recordings_path, subject, '%s-%s-%s-%s-fwd.fif' % (
            surface, spacing, add_dist, src_type)), 
    out['fwd'], overwrite=True)


mne.write_source_spaces(
    op.join(recordings_path, subject, '%s-%s-%s-%s-src.fif' % (
            surface, spacing, add_dist, src_type)), 
    out['src_subject'])

fname_bem = op.join(
    subjects_dir, subject, 'bem', '%s-%i-bem.fif' % (
    subject, out['bem_sol']['solution'].shape[0]))


Overwriting existing file.
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


In [44]:
def make_fwd_stack(subject, subjects_dir, hcp_path, recordings_path,
                   surface='white', add_dist=True,
                   src_type='subject_on_fsaverage',
                   spacings=('oct6', 'oct5', 'oct4', 'ico4')):

    hcp.make_mne_anatomy(subject=subject, subjects_dir=subjects_dir, hcp_path=hcp_path,
                         recordings_path=recordings_path)

    for spacing in spacings:
        out = hcp.anatomy.compute_forward_stack(
            subjects_dir=subjects_dir, subject=subject,
            recordings_path=recordings_path,
            src_params=dict(spacing=spacing,
                            add_dist=add_dist, surface=surface),
            hcp_path=hcp_path)

        mne.write_forward_solution(
            op.join(recordings_path, subject, '%s-%s-%s-%s-fwd.fif' % (
                    surface, spacing, add_dist, src_type)), 
            out['fwd'], overwrite=True)

        mne.write_source_spaces(
            op.join(recordings_path, subject, '%s-%s-%s-%s-src.fif' % (
                    surface, spacing, add_dist, src_type)), 
            out['src_subject'])

        fname_bem = op.join(
            subjects_dir, subject, 'bem', '%s-%i-bem.fif' % (
            subject, out['bem_sol']['solution'].shape[0]))
        mne.write_bem_solution(
            fname_bem, out['bem'])

preprare_inverse(subject=subject, subjects_dir=subjects_dir, hcp_path=hcp_path,
                 recordings_path=recordings_path,)

reading extended structural processing ...
reading RAS freesurfer transform
Combining RAS transform and coregistration
extracting head model
coregistring head model to MNE-HCP coordinates
extracting coregistration
Setting up the source space with the following parameters:

SUBJECTS_DIR = /home/ubuntu/mne-hcp-data/hcp-subjects
Subject      = fsaverage
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading /home/ubuntu/mne-hcp-data/hcp-subjects/fsaverage/surf/lh.white...
Triangle file: created by nicks on Tue Aug 14 13:43:20 2007 nvert = 163842 ntri = 327680
    Triangle neighbors and vertex normals...
Loading geometry from /home/ubuntu/mne-hcp-data/hcp-subjects/fsaverage/surf/lh.sphere...
Triangle file: created by greve on Thu Jun  8 19:17:51 2006 nvert = 163842 ntri = 327680
Mapping lh fsaverage -> oct (6) ...
Setting up the triangulation for the decimated surface...
loaded lh.white 4098/163842 selected t

In [ ]:
def compute_inverse_operator(subject, recordings_path, data_type, run_index=0,
                             cov_method='empirical', surf='white', spacing='oct6', add_dist=True,
                             src_type='subject_on_fsaverage', fmin=):

    fwd_fname = op.join(
        recordings_path, subject, '%s-%s-%s-%s-fwd.fif' % (
            surf, spacing, add_dist, src_type))

    fwd = mne.read_forward_solution(fwd_fname)
    data_type = 'noise_empty_room'
    
    if any([fmin, fmax]):
        hcp_band_pass(this_raw, fmin=fmin, fmax=fmax, notch=False)
        fcomment = 'lp{}-hp-{}'.format(
            fmin if fmin is None else str(fmin).replace('.', 'p'),
            fmax if fmax is None else str(fmax).replace('.', 'p'))
    else:
        fcomment = 'broad'
    

    noise_cov = mne.read_cov(
        op.join(recordings_path, subject, '%s-%s-%s-cov.fif' % (
                data_type, cov_method, fcomment)))

    raw = mne.io.read_raw_fif(
        op.join(recordings_path, subject, '%s-run%i-preproc-raw.fif' % (
                data_type, run_index)))

    inverse_operator = mne.minimum_norm.make_inverse_operator(
        info=raw.info, noise_cov=noise_cov, forward=fwd)
    
    mne.minimum_norm.write_inverse_operator(
        fname=fwd_fname.replace('fwd', 'inv'), inv=inverse_operator)
    return inverse_operator

compute_inverse_operator(hcp_params['subject'],
                         hcp_params['recordings_path'],
                         hcp_params['data_type'])

In [ ]:
#     hcp_params = dict(subject=subject, hcp_path=hcp_path,
#                   data_type=data_type)

#     out_path = op.join(recordings_path, subject)

#     for run_index in run_inds:
#         hcp_params['run_index'] = run_index
#         raw = io.read_raw_hcp(**hcp_params)
#         raw.load_data()
    
#         raw = _preprocess_raw(raw, hcp_params, ica_sel='brain')
        
#         duration = n_fft * (1 / raw.info['sfreq'])
#         events = mne.make_fixed_length_events(raw, 42, duration=duration)
#         epochs = mne.Epochs(raw, events=events, event_id=42, tmin=0, tmax=duration,
#                             baseline=None, preload=False)
